## Question 2
### Steps
1. Load the datasets
2. Filter out group 4 States
3. Merge the dental benefit database with enrollment database
4. Report by States
5. Report by top-5 insurers

#### 1. Load the datasets

In [1]:
# keep the leading zeros in column `Plan ID`
import pandas as pd
enrollment = pd.read_csv('/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW2/CPSC_Enrollment_2022_01/CPSC_Enrollment_Info_2022_01.csv')

In [2]:
plan = pd.read_excel('/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW2/Monthly_Report_By_Plan_2022_01/Monthly_Report_By_Plan_2022_01.xlsx',header=5,skipfooter=3,)
plan.head()

/Users/cynding/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Contract Number,Plan ID,Organization Type,Plan Type,Offers Part D,Organization Name,Organization Marketing Name,Plan Name,Parent Organization,Contract Effective Date,Enrollment
0,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,No,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,United Mine Workers of America Health & Retire...,NaN,UMWA Health and Retirement Funds,1974-02-01 00:00:00,44280
1,H0022,1,Demo,Medicare-Medicaid Plan HMO,Yes,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Buckeye Health Plan - MyCare Ohio,Buckeye Health Plan - MyCare Ohio (Medicare-Me...,Centene Corporation,2014-05-01 00:00:00,15752
2,H0028,7,Local CCP,HMO,Yes,"CHA HMO, INC.",Humana,Humana Gold Plus SNP-DE H0028-007 (HMO D-SNP),Humana Inc.,2013-01-01 00:00:00,686
3,H0028,13,Local CCP,HMO,Yes,"CHA HMO, INC.",Humana,Humana Gold Plus H0028-013 (HMO),Humana Inc.,2013-01-01 00:00:00,2984
4,H0028,14,Local CCP,HMO,Yes,"CHA HMO, INC.",Humana,Humana Gold Plus H0028-014 (HMO),Humana Inc.,2013-01-01 00:00:00,14941


In [3]:
# keep the necessary columns
plan = plan[['Contract Number','Plan ID','Organization Name']]

In [4]:
dental = pd.read_table('/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW2/PBP_Benefits_2021 07_01_2021/pbp_b16_dental.txt')

/Users/cynding/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (98) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### 2. Filter out group 4 States

In [5]:
State_list = ['HI','MI','MN','MS','NY','OK','SD','TN']
enrollment = enrollment[enrollment['State'].isin(State_list)]

#### 3. Merge the dental benefit database with enrollment database

In [6]:
# only keep the necessary columns
dental = dental[['pbp_a_hnumber','pbp_a_plan_identifier','segment_id','pbp_b16a_bendesc_yn','pbp_b16b_bendesc_yn']]

In [7]:
temp = pd.merge(enrollment[['Contract Number','Plan ID','State']],dental,how='inner',left_on=['Plan ID','Contract Number'], right_on=['pbp_a_plan_identifier','pbp_a_hnumber'])
temp.head()

,Contract Number,Plan ID,State,pbp_a_hnumber,pbp_a_plan_identifier,segment_id,pbp_b16a_bendesc_yn,pbp_b16b_bendesc_yn
0,H0022,1,HI,H0022,1,0,1.0,1.0
1,H0022,1,MI,H0022,1,0,1.0,1.0
2,H0022,1,MI,H0022,1,0,1.0,1.0
3,H0022,1,MI,H0022,1,0,1.0,1.0
4,H0022,1,MI,H0022,1,0,1.0,1.0


#### Then merge our temporary dataframe with plan database to retrieve the insurer info. 

In [8]:
df = pd.merge(temp,plan,how='inner',on=['Contract Number','Plan ID'])

In [9]:
df.head()

,Contract Number,Plan ID,State,pbp_a_hnumber,pbp_a_plan_identifier,segment_id,pbp_b16a_bendesc_yn,pbp_b16b_bendesc_yn,Organization Name
0,H0022,1,HI,H0022,1,0,1.0,1.0,"BUCKEYE COMMUNITY HEALTH PLAN, INC."
1,H0022,1,MI,H0022,1,0,1.0,1.0,"BUCKEYE COMMUNITY HEALTH PLAN, INC."
2,H0022,1,MI,H0022,1,0,1.0,1.0,"BUCKEYE COMMUNITY HEALTH PLAN, INC."
3,H0022,1,MI,H0022,1,0,1.0,1.0,"BUCKEYE COMMUNITY HEALTH PLAN, INC."
4,H0022,1,MI,H0022,1,0,1.0,1.0,"BUCKEYE COMMUNITY HEALTH PLAN, INC."


In [10]:
# the overall percentage of enrollees enjoying the “Preventive Dental Items as a supplemental benefit under Part C”?
df[df['pbp_b16a_bendesc_yn']==1.0].shape[0]/df.shape[0]

0.09264531747493619

In [11]:
# the overall percentage of enrollees enjoying the “Comprehensive Dental Items as a supplemental benefit under Part C”?
df[df['pbp_b16b_bendesc_yn']==1.0].shape[0]/df.shape[0]

0.08729310843880746

#### 4. Report by States

In [12]:
p_dental = df.groupby([df['State'],df['pbp_b16a_bendesc_yn']])['Contract Number'].count().unstack(level=1)

In [13]:
p_dental_perc = p_dental[1.0].div(p_dental.sum(axis=1),axis=0)

In [14]:
c_dental = df.groupby([df['State'],df['pbp_b16b_bendesc_yn']])['Contract Number'].count().unstack(level=1)

In [15]:
c_dental_perc = c_dental[1.0].div(c_dental.sum(axis=1),axis=0)

In [16]:
by_states = pd.DataFrame(dict(p_dental_perc = p_dental_perc, c_dental_perc = c_dental_perc))

In [17]:
by_states.style.format({
    'p_dental_perc': '{:,.2%}'.format,
    'c_dental_perc': '{:,.2%}'.format,
})

,p_dental_perc,c_dental_perc
State,,
HI,11.77%,10.33%
MI,15.95%,15.14%
MN,8.17%,7.95%
MS,5.80%,5.64%
NY,14.06%,12.53%
OK,5.78%,5.20%
SD,3.30%,3.22%
TN,9.87%,9.51%


#### 5. Report by top-5 insurers

#### Find the top-5 insurers

In [34]:
temp = df.groupby(df['Organization Name']).count().sort_values(by='Contract Number',ascending=False)
top_5 = temp.iloc[:5,:]
top_5 = top_5.index.to_list()
print(top_5)

['CARE IMPROVEMENT PLUS SOUTH CENTRAL INSURANCE CO.', 'SIERRA HEALTH AND LIFE INSURANCE COMPANY, INC.', 'ANTHEM INSURANCE COMPANIES, INC.', 'HUMANA INSURANCE COMPANY', 'AETNA LIFE INSURANCE COMPANY']


In [19]:
df = df[df['Organization Name'].isin(top_5)]

In [20]:
p_dental = df.groupby([df['Organization Name'],df['pbp_b16a_bendesc_yn']])['Contract Number'].count().unstack(level=1)

In [21]:
p_dental_perc = p_dental[1.0].div(p_dental.sum(axis=1),axis=0)

In [22]:
c_dental = df.groupby([df['Organization Name'],df['pbp_b16b_bendesc_yn']])['Contract Number'].count().unstack(level=1)

In [23]:
c_dental_perc = c_dental[1.0].div(c_dental.sum(axis=1),axis=0)

In [24]:
by_insurers = pd.DataFrame(dict(p_dental_perc = p_dental_perc, c_dental_perc = c_dental_perc))

In [25]:
by_insurers.style.format({
    'p_dental_perc': '{:,.2%}'.format,
    'c_dental_perc': '{:,.2%}'.format,
})

,p_dental_perc,c_dental_perc
Organization Name,,
AETNA LIFE INSURANCE COMPANY,20.31%,20.29%
"ANTHEM INSURANCE COMPANIES, INC.",0.44%,0.15%
CARE IMPROVEMENT PLUS SOUTH CENTRAL INSURANCE CO.,1.36%,1.11%
HUMANA INSURANCE COMPANY,39.66%,39.15%
"SIERRA HEALTH AND LIFE INSURANCE COMPANY, INC.",0.11%,0.10%
